### Improving Runtime of the Code

1. Imploy the coin flipping way to spread influence to we don't need to have random variable set up repetitively for an edge.
(This actually don't offer much performance improvement as in the original logic, we also only spread on one link once) But this way the code is overall clearner and there might be less overhead.
2. Delete the history variable so that we don't need to store every timestamp as we don't need to plot in this case
3. Decrease the simulation run for each choice in greedy algorithm

In [1]:
import networkx as nx
import random

class InfluenceDeinfluenceModel:
    def __init__(self, graph):
        self.graph = graph
        self.init_edge_weights()
        self.activated_edges = set()

    def init_edge_weights(self):
        for u, v in self.graph.edges:
            p_is = random.uniform(0, 1)
            p_ds = random.uniform(0, 1)
            p_di = random.uniform(0, 1)
            self.graph[u][v]['p_is'] = p_is
            self.graph[u][v]['p_ds'] = p_ds
            self.graph[u][v]['p_di'] = p_di

    def set_initial_states(self):
        nx.set_node_attributes(self.graph, 'S', 'state')

    def set_influencers(self, influencers):
        for node in influencers:
            self.graph.nodes[node]['state'] = 'I'

    def set_deinfluencers(self, deinfluencers):
        for node in deinfluencers:
            self.graph.nodes[node]['state'] = 'D'

    def pre_determine_active_edges(self):
        self.active_edges = set()
        for node in self.graph.nodes:
            if self.graph.nodes[node]['state'] == 'I':
                for neighbor in self.graph.neighbors(node):
                    if (self.graph.nodes[neighbor]['state'] == 'S' and 
                        random.random() < self.graph[node][neighbor]['p_is']):
                        self.active_edges.add((node, neighbor))
            elif self.graph.nodes[node]['state'] == 'D':
                for neighbor in self.graph.neighbors(node):
                    if self.graph.nodes[neighbor]['state'] == 'S' and random.random() < self.graph[node][neighbor]['p_ds']:
                        self.active_edges.add((node, neighbor))
                    elif self.graph.nodes[neighbor]['state'] == 'I' and random.random() < self.graph[node][neighbor]['p_di']:
                        self.active_edges.add((node, neighbor))


    def spread_influence(self):
        new_influenced = set()
        new_deinfluenced = set()

        for edge in self.active_edges:
            node, neighbor = edge
            if self.graph.nodes[node]['state'] == 'I' and self.graph.nodes[neighbor]['state'] == 'S':
                new_influenced.add(neighbor)
            elif self.graph.nodes[node]['state'] == 'D':
                if self.graph.nodes[neighbor]['state'] == 'S':
                    new_deinfluenced.add(neighbor)
                elif self.graph.nodes[neighbor]['state'] == 'I':
                    new_deinfluenced.add(neighbor)

        for node in new_influenced:
            self.graph.nodes[node]['state'] = 'I'
        for node in new_deinfluenced:
            self.graph.nodes[node]['state'] = 'D'

    def influencer_spread_influence(self):
        new_influenced = set()
        for edge in self.active_edges:
            node, neighbor = edge
            if self.graph.nodes[node]['state'] == 'I' and self.graph.nodes[neighbor]['state'] == 'S':
                new_influenced.add(neighbor)

        for node in new_influenced:
            self.graph.nodes[node]['state'] = 'I'

    def run_cascade(self, steps):
        for _ in range(steps):
            self.pre_determine_active_edges()
            self.spread_influence()

    def run_cascade_influencer(self, steps):
        for _ in range(steps):
            self.pre_determine_active_edges()
            self.influencer_spread_influence()

    def evaluate_influence(self):
        """Evaluate the number of influenced nodes."""
        return sum(1 for node in self.graph.nodes if self.graph.nodes[node]['state'] == 'I')

    def evaluate_deinfluence(self):
        return sum(1 for node in self.graph.nodes if self.graph.nodes[node]['state'] == 'D')

    def greedy_hill_climbing(self, k, steps, R=10):
        """Select k initial influencers using the improved greedy algorithm."""
        best_influencers = set()

        for _ in range(k):
            best_candidate = None
            best_score = -1

            for node in self.graph.nodes:
                if node in best_influencers:
                    continue
                
                # Temporarily add the candidate node to the set of influencers
                current_influencers = best_influencers | {node}
                total_score = 0

                for _ in range(R):
                    self.activated_edges.clear()  # Reset activated edges
                    self.set_initial_states()
                    self.set_influencers(current_influencers)
                    self.run_cascade_influencer(steps)
                    total_score += self.evaluate_influence()

                avg_score = total_score / R

                if avg_score > best_score:
                    best_score = avg_score
                    best_candidate = node

            if best_candidate is not None:
                best_influencers.add(best_candidate)

        return best_influencers

    def generate_random_graph(self, p):
        random_graph = nx.DiGraph()
        for u, v in self.graph.edges:
            if random.random() < p:
                random_graph.add_edge(u, v)
        return random_graph

    def compute_reachable_set(self, graph, seeds):
        reachable = set(seeds)
        to_explore = list(seeds)
        while to_explore:
            current = to_explore.pop()
            if current in graph:  # Ensure current node exists in the graph
                for neighbor in graph.neighbors(current):
                    if neighbor not in reachable:
                        reachable.add(neighbor)
                        to_explore.append(neighbor)
        return reachable

    def greedy_hill_climbing_new(self, k, R=20000, p=0.1):
        """Select k initial influencers using the NewGreedyIC algorithm."""
        S = []
        for _ in range(k):
            sv = {v: 0 for v in self.graph if v not in S}
            for _ in range(R):
                G_prime = self.generate_random_graph(p)
                reachable_S = self.compute_reachable_set(G_prime, S)
                for v in sv:
                    if v not in reachable_S:
                        reachable_v = self.compute_reachable_set(G_prime, {v})
                        sv[v] += len(reachable_v)
            for v in sv:
                sv[v] /= R
            best_candidate = max(sv, key=sv.get)
            S.append(best_candidate)
        return S

    def reset_graph(self):
        self.set_initial_states()
        self.activated_edges = set()


In [2]:
# Example usage
num_nodes = 200
steps = 10
edge_prob = 0.2
num_influencers = 5

def generate_and_run_model(graph_type, num_nodes, steps, edge_prob=None, k=None, p=None, m=None, num_influencers=3):
    if graph_type == 'erdos_renyi':
        graph = nx.erdos_renyi_graph(num_nodes, edge_prob, directed=True)
    elif graph_type == 'watts_strogatz':
        graph = nx.watts_strogatz_graph(num_nodes, k, p).to_directed()
    elif graph_type == 'barabasi_albert':
        graph = nx.barabasi_albert_graph(num_nodes, m).to_directed()
    else:
        raise ValueError("Unsupported graph type. Choose from 'erdos_renyi', 'watts_strogatz', or 'barabasi_albert'.")

    model = InfluenceDeinfluenceModel(graph)
    model.set_initial_states()
    initial_influencers = model.greedy_hill_climbing(num_influencers, steps)
    #initial_influencers = model.greedy_hill_climbing_new(num_influencers)
    print("Optimized Initial Influencers:", initial_influencers)
    
    model.set_initial_states()
    model.set_influencers(initial_influencers)
    model.set_deinfluencers([2])  # Example: Set a fixed deinfluencer
    model.run_cascade(steps)

    return model
    
model_er = generate_and_run_model('barabasi_albert', num_nodes, steps, edge_prob=edge_prob, num_influencers=num_influencers, m=2)

print("count deinfuence",model_er.evaluate_deinfluence())

Optimized Initial Influencers: {0, 102, 167, 140, 25}
count deinfuence 198


Key Changes:

Parallel Processing:Used the joblib library to parallelize the simulations in the greedy_hill_climbing and greedy_hill_climbing_new methods. This leverages multiple CPU cores to speed up the calculations.

Efficient Data Structures:Used sets and dictionaries for fast lookups and updates.

Batch Processing: Updated the code to process nodes in batches where feasible to reduce the number of iterations.

In [7]:
from joblib import Parallel, delayed
import networkx as nx
import random

class InfluenceDeinfluenceModel:
    def __init__(self, graph):
        self.graph = graph
        self.init_edge_weights()
        self.activated_edges = set()

    def init_edge_weights(self):
        for u, v in self.graph.edges:
            p_is = random.uniform(0, 1)
            p_ds = random.uniform(0, 1)
            p_di = random.uniform(0, 1)
            self.graph[u][v]['p_is'] = p_is
            self.graph[u][v]['p_ds'] = p_ds
            self.graph[u][v]['p_di'] = p_di

    def set_initial_states(self):
        nx.set_node_attributes(self.graph, 'S', 'state')

    def set_influencers(self, influencers):
        for node in influencers:
            self.graph.nodes[node]['state'] = 'I'

    def set_deinfluencers(self, deinfluencers):
        for node in deinfluencers:
            self.graph.nodes[node]['state'] = 'D'

    def pre_determine_active_edges(self):
        self.active_edges = set()
        for node in self.graph.nodes:
            if self.graph.nodes[node]['state'] == 'I':
                for neighbor in self.graph.neighbors(node):
                    if (self.graph.nodes[neighbor]['state'] == 'S' and 
                        random.random() < self.graph[node][neighbor]['p_is']):
                        self.active_edges.add((node, neighbor))
            elif self.graph.nodes[node]['state'] == 'D':
                for neighbor in self.graph.neighbors(node):
                    if self.graph.nodes[neighbor]['state'] == 'S' and random.random() < self.graph[node][neighbor]['p_ds']:
                        self.active_edges.add((node, neighbor))
                    elif self.graph.nodes[neighbor]['state'] == 'I' and random.random() < self.graph[node][neighbor]['p_di']:
                        self.active_edges.add((node, neighbor))


    def spread_influence(self):
        new_influenced = set()
        new_deinfluenced = set()

        for edge in self.active_edges:
            node, neighbor = edge
            if self.graph.nodes[node]['state'] == 'I' and self.graph.nodes[neighbor]['state'] == 'S':
                new_influenced.add(neighbor)
            elif self.graph.nodes[node]['state'] == 'D':
                if self.graph.nodes[neighbor]['state'] == 'S':
                    new_deinfluenced.add(neighbor)
                elif self.graph.nodes[neighbor]['state'] == 'I':
                    new_deinfluenced.add(neighbor)

        for node in new_influenced:
            self.graph.nodes[node]['state'] = 'I'
        for node in new_deinfluenced:
            self.graph.nodes[node]['state'] = 'D'

    def influencer_spread_influence(self):
        new_influenced = set()
        for edge in self.active_edges:
            node, neighbor = edge
            if self.graph.nodes[node]['state'] == 'I' and self.graph.nodes[neighbor]['state'] == 'S':
                new_influenced.add(neighbor)

        for node in new_influenced:
            self.graph.nodes[node]['state'] = 'I'

    def run_cascade(self, steps):
        for _ in range(steps):
            self.pre_determine_active_edges()
            self.spread_influence()

    def run_cascade_influencer(self, steps):
        for _ in range(steps):
            self.pre_determine_active_edges()
            self.influencer_spread_influence()

    def evaluate_influence(self):
        """Evaluate the number of influenced nodes."""
        return sum(1 for node in self.graph.nodes if self.graph.nodes[node]['state'] == 'I')

    def evaluate_deinfluence(self):
        return sum(1 for node in self.graph.nodes if self.graph.nodes[node]['state'] == 'D')
    
    def greedy_hill_climbing(self, k, steps, R=10):
        #Select k initial influencers using the improved greedy algorithm.
        best_influencers = set()

        def simulate_influence(current_influencers):
            total_score = 0
            for _ in range(R):
                self.reset_graph()
                self.set_influencers(current_influencers)
                self.run_cascade_influencer(steps)
                total_score += self.evaluate_influence()
            return total_score / R

        for _ in range(k):
            best_candidate = None
            best_score = -1

            candidates = [node for node in self.graph.nodes if node not in best_influencers]
            scores = Parallel(n_jobs=-1)(delayed(simulate_influence)(best_influencers | {node}) for node in candidates)

            best_score, best_candidate = max(zip(scores, candidates))

            if best_candidate is not None:
                best_influencers.add(best_candidate)

        return best_influencers

    def reset_graph(self):
        self.set_initial_states()
        self.activated_edges = set()

In [8]:
# Example usage
num_nodes = 500
steps = 3
edge_prob = 0.2
num_influencers = 3

def generate_and_run_model(graph_type, num_nodes, steps, edge_prob=None, k=None, p=None, m=None, num_influencers=3):
    if graph_type == 'erdos_renyi':
        graph = nx.erdos_renyi_graph(num_nodes, edge_prob, directed=True)
    elif graph_type == 'watts_strogatz':
        graph = nx.watts_strogatz_graph(num_nodes, k, p).to_directed()
    elif graph_type == 'barabasi_albert':
        graph = nx.barabasi_albert_graph(num_nodes, m).to_directed()
    else:
        raise ValueError("Unsupported graph type. Choose from 'erdos_renyi', 'watts_strogatz', or 'barabasi_albert'.")

    model = InfluenceDeinfluenceModel(graph)
    model.set_initial_states()
    initial_influencers = model.greedy_hill_climbing(num_influencers, steps)
    #initial_influencers = model.greedy_hill_climbing_new(num_influencers)
    print("Optimized Initial Influencers:", initial_influencers)
    
    model.set_initial_states()
    model.set_influencers(initial_influencers)
    model.set_deinfluencers([2])  # Example: Set a fixed deinfluencer
    model.run_cascade(steps)

    return model
    
model = generate_and_run_model('barabasi_albert', num_nodes, steps, edge_prob=edge_prob, num_influencers=num_influencers, m=2)

print("count infuence",model.evaluate_deinfluence())

Optimized Initial Influencers: {2, 4, 12}
count infuence 219
